In [1]:
"""
Some results (e.g. miconic evaluation, or blocksworld training) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies, or very large training datasets).
"""

from _results_helper import *

/home/dzc/code/cvut-colab/results/baseline_logs


/home/dzc/code/cvut-colab/results/baseline_logs/transport_2_30_r9.log: 100%|██████████| 4500/4500 [00:01<00:00, 3715.89it/s] 


skipping hard problems because they take too long


# Plan length plots

In [2]:
log_y = 0
include_sample = 0
dimensions = [4, 8, 16, 32, 64]

plot_domains("plan_length", log_y=log_y, include_sample=include_sample, dimensions=dimensions)

blocksworld


ferry


miconic


satellite


transport


In [3]:
# plot_domains("time")

# Absolute difference plots

In [4]:
include_sample = 0
dimensions = [4, 8, 16, 32, 64]

plot_difference(absolute=True, include_sample=include_sample, dimensions=dimensions)

blocksworld


ferry


# Relative difference plots

In [ ]:
include_sample = 0
dimensions = [4, 8, 16, 32, 64]

plot_difference(absolute=False, include_sample=include_sample, dimensions=dimensions)

blocksworld


ferry


miconic


satellite


transport
